In [1]:
!pip install scrapy
!pip install crochet

  Using cached Scrapy-2.12.0-py2.py3-none-any.whl (311 kB)
  Using cached lxml-5.3.0-cp310-cp310-manylinux_2_28_x86_64.whl (5.0 MB)
  Using cached itemloaders-1.3.2-py3-none-any.whl (12 kB)
  Using cached itemadapter-0.10.0-py3-none-any.whl (11 kB)
  Using cached cssselect-1.2.0-py2.py3-none-any.whl (18 kB)
  Using cached tldextract-5.1.3-py3-none-any.whl (104 kB)
  Using cached Protego-0.3.1-py2.py3-none-any.whl (8.5 kB)
  Using cached service_identity-24.2.0-py3-none-any.whl (11 kB)
  Using cached parsel-1.9.1-py2.py3-none-any.whl (17 kB)
  Using cached twisted-24.11.0-py3-none-any.whl (3.2 MB)
  Using cached zope.interface-7.2-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (254 kB)
  Using cached PyDispatcher-2.0.7-py3-none-any.whl (12 kB)
  Using cached queuelib-1.7.0-py2.py3-none-any.whl (13 kB)
  Using cached cryptography-44.0.0-cp39-abi3-manylinux_2_28_x86_64.whl (4.2 MB)
  Using cached pyOpenSSL-24.3.0-py3-none-any.whl (56 kB)


In [6]:
%reset -sf
%reset -sf

In [3]:
import scrapy

class QuotesSpider(scrapy.Spider):
    name = 'quotes'
    allowed_domains = ['quotes.toscrape.com']
    start_urls = ['https://quotes.toscrape.com']

    async def parse(self, response):
        page = response.css('div.quote')
        for element in page:
            text = element.css("span.text::text").get()
            yield { 'text': text }

        next_page = response.css('li.next a::attr(href)').get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            self.log(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

In [4]:
%kernelspec list

UsageError: Line magic function `%kernelspec` not found.


In [7]:
import crochet
from scrapy.crawler import CrawlerProcess
from twisted.internet import reactor


crochet.setup()

@crochet.run_in_reactor
def run_spider():
    process = CrawlerProcess(
        settings={
            'FEEDS': {'output.json': {'format': 'json'}},
            'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        },
    )
    process.crawl(QuotesSpider)
    process.start()

run_spider()


ReactorStopped: 